<a href="https://colab.research.google.com/github/iam-fern/Generate_data/blob/main/ex_sampling_c_1000rep_n30_x4_sas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

import statistics as stats
import scipy.stats as ss
import scipy.stats as st
from scipy.stats import f_oneway
from scipy import stats
import scipy.stats as stats
import time
import statsmodels.api as sm

from statsmodels.formula.api import ols

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df = pd.read_csv('https://github.com/iam-fern/Research/raw/main/sampling_1000rep_n30_x4_sas.csv', index_col=['rep', 'index'])
df

x1         x2  ...         y_hat         e_hat
rep   index                        ...                            
0.0   0.0    44.119476  46.126732  ... -11167.803984 -5.463866e+06
      1.0    24.587075  34.682826  ...  -2650.773987 -5.358138e+05
      2.0    33.144946  48.714841  ... -11874.790962 -2.505356e+06
      3.0    42.618377  37.883982  ...  18965.253271 -4.753369e+06
      4.0    48.161847  48.492968  ...  -3870.899363  2.577689e+06
...                ...        ...  ...           ...           ...
999.0 25.0   30.905022  39.363861  ...  20654.215078 -8.269774e+05
      26.0   26.538237  13.278744  ...  24009.488106  3.793522e+05
      27.0   17.923933  20.276218  ...  -8387.248911 -9.159291e+04
      28.0   32.630332  48.709353  ... -25550.710020  1.579141e+05
      29.0   12.188519  30.109073  ...   6193.686670  3.828672e+04

[30000 rows x 7 columns]

In [3]:
# ลบข้อมูลที่มี missing values
table = df.dropna()
table

x1         x2  ...         y_hat         e_hat
rep   index                        ...                            
0.0   0.0    44.119476  46.126732  ... -11167.803984 -5.463866e+06
      1.0    24.587075  34.682826  ...  -2650.773987 -5.358138e+05
      2.0    33.144946  48.714841  ... -11874.790962 -2.505356e+06
      3.0    42.618377  37.883982  ...  18965.253271 -4.753369e+06
      4.0    48.161847  48.492968  ...  -3870.899363  2.577689e+06
...                ...        ...  ...           ...           ...
999.0 25.0   30.905022  39.363861  ...  20654.215078 -8.269774e+05
      26.0   26.538237  13.278744  ...  24009.488106  3.793522e+05
      27.0   17.923933  20.276218  ...  -8387.248911 -9.159291e+04
      28.0   32.630332  48.709353  ... -25550.710020  1.579141e+05
      29.0   12.188519  30.109073  ...   6193.686670  3.828672e+04

[30000 rows x 7 columns]

In [4]:
table.reset_index(inplace=True)
table

,rep,index,x1,x2,x3,e,Y,y_hat,e_hat
0,0.0,0.0,44.119476,46.126732,12.656551,-5.475539e+06,-5.475034e+06,-11167.803984,-5.463866e+06
1,0.0,1.0,24.587075,34.682826,30.125790,-5.387803e+05,-5.384646e+05,-2650.773987,-5.358138e+05
2,0.0,2.0,33.144946,48.714841,27.271987,-2.517643e+06,-2.517231e+06,-11874.790962,-2.505356e+06
3,0.0,3.0,42.618377,37.883982,33.475661,-4.734906e+06,-4.734404e+06,18965.253271,-4.753369e+06
4,0.0,4.0,48.161847,48.492968,17.519104,2.573266e+06,2.573819e+06,-3870.899363,2.577689e+06
...,...,...,...,...,...,...,...,...,...
29995,999.0,25.0,30.905022,39.363861,49.948571,-8.067266e+05,-8.063232e+05,20654.215078,-8.269774e+05
29996,999.0,26.0,26.538237,13.278744,29.277006,4.030488e+05,4.033617e+05,24009.488106,3.793522e+05
29997,999.0,27.0,17.923933,20.276218,16.417689,-1.002011e+05,-9.998016e+04,-8387.248911,-9.159291e+04
29998,999.0,28.0,32.630332,48.709353,15.047005,1.319683e+05,1.323634e+05,-25550.710020,1.579141e+05


In [5]:
table.set_index(['rep', 'index'], inplace=True)

In [6]:
table

x1         x2  ...         y_hat         e_hat
rep   index                        ...                            
0.0   0.0    44.119476  46.126732  ... -11167.803984 -5.463866e+06
      1.0    24.587075  34.682826  ...  -2650.773987 -5.358138e+05
      2.0    33.144946  48.714841  ... -11874.790962 -2.505356e+06
      3.0    42.618377  37.883982  ...  18965.253271 -4.753369e+06
      4.0    48.161847  48.492968  ...  -3870.899363  2.577689e+06
...                ...        ...  ...           ...           ...
999.0 25.0   30.905022  39.363861  ...  20654.215078 -8.269774e+05
      26.0   26.538237  13.278744  ...  24009.488106  3.793522e+05
      27.0   17.923933  20.276218  ...  -8387.248911 -9.159291e+04
      28.0   32.630332  48.709353  ... -25550.710020  1.579141e+05
      29.0   12.188519  30.109073  ...   6193.686670  3.828672e+04

[30000 rows x 7 columns]

In [7]:
def get_result(run_replicate):
    new_table = table.loc[run_replicate]

    # show index
    new_table.reset_index(inplace=True)

    new_table.drop(['index'], axis=1, inplace=True)


    # แบ่งข้อมูลออกเป็น 2 กลุ่มด้วย k-mean

    #Set number of clusters
    x = new_table[["x1", "x2", "x3"]]
    k_means = KMeans(n_clusters=2)

    #Run the clustering algorithm
    model = k_means.fit(x)
    model

    #Generate cluster predictions and store in y_kmeans
    y_kmeans = k_means.predict(x)
    y_kmeans

    # kmean_clusters.astype(float)
    kmean_clusters = pd.DataFrame({"clusters" : y_kmeans})

    # เชื่อมข้อมูลทั้ง 2 DataFrame เข้าด้วยกันด้วยคำสั่ง pd.concat
    data_clusters = pd.concat([new_table, kmean_clusters], axis=1)

    data_sort_clusters = data_clusters.sort_values('clusters')

    # print(data_clusters.groupby('clusters').groups)

    data_sort_clusters.reset_index(inplace=True)

    # กลุ่มที่ 1
    data_cluster1 = data_sort_clusters[data_sort_clusters.clusters == 1]

    y = data_cluster1[['Y']]
    x = data_cluster1[['x1', 'x2', 'x3']]
    x = sm.add_constant(x)
    model1 = sm.OLS(y, x).fit()

    DF1 = model1.df_resid

    SSR1 = model1.ssr

    MSE1 = model1.mse_resid

    # กลุ่มที่ 2
    data_cluster0 = data_sort_clusters[data_sort_clusters.clusters == 0]

    y = data_cluster0[['Y']]
    x = data_cluster0[['x1', 'x2', 'x3']]
    x = sm.add_constant(x)
    model0 = sm.OLS(y, x).fit()

    DF0 = model0.df_resid

    SSR0 = model0.ssr

    MSE0 = model0.mse_resid

    df1 = DF1+1
    df0 = DF0+1

    # คำนวณค่าสถิติ
    GQ_test_clusters = (SSR1/df1) / (SSR0/df0)


    # คำนวณค่า p-value
    # sf(x, dfn, dfd, loc=0, scale=1) : Survival function (also defined as 1 - cdf, but sf is sometimes more accurate)
    two_tailed = 2.0*(1.0-(ss.f.cdf(GQ_test_clusters, df1, df0)))

    # pdf(x, dfn, dfd, loc=0, scale=1) : Probability density function
    p_greater = ss.f.cdf(GQ_test_clusters, df1, df0)
    
    return df0, df1, SSR0, SSR1, MSE0, MSE1, GQ_test_clusters, two_tailed, p_greater

# สร้าง list ของ replicates
list_replicate=list({i[0] for i in table.index})
list_replicate.sort()

# สร้าง list เพื่อเก็บค่า ทุกรอบที่ run
list_df0=[]
list_df1=[]
list_SSR0=[]
list_SSR1=[]
list_MSE0=[]
list_MSE1=[]
list_GQ_test_clusters=[]
list_result_test=[]
list_two_tailed=[]
list_p_greater=[]
list_result_test01=[]

# ใช้ for loop สำหรับการ run ทุก replicates
for run_replicate in list_replicate:
    df0, df1, SSR0, SSR1, MSE0, MSE1, GQ_test_clusters, two_tailed, p_greater=get_result(run_replicate)

    list_df0.append(df0)
    list_df1.append(df1)
    list_SSR0.append(SSR0)
    list_SSR1.append(SSR1)
    list_MSE0.append(MSE0)
    list_MSE1.append(MSE1)
    list_GQ_test_clusters.append(GQ_test_clusters)
    list_two_tailed.append(two_tailed)
    list_p_greater.append(p_greater)

In [8]:
# สร้างตารางผลลัพธ์
alpha = 0.05
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
two_tailed_new=pd.DataFrame({"Replicate":list_replicate, "df0":list_df0, "df1":list_df1, "SSR0":list_SSR0,
                     "SSR1":list_SSR1, "MSE0":list_MSE0, "MSE1":list_MSE1, 
                     "GQ_test_clusters":list_GQ_test_clusters, "two_tailed":list_two_tailed, "greater":list_p_greater})
two_tailed_new.set_index(['Replicate'], inplace=True)
two_tailed_new=two_tailed_new[(two_tailed_new["two_tailed"]<=1) & (two_tailed_new["greater"]<=1)]
two_tailed_new["Result_test"]=two_tailed_new["two_tailed"].apply(lambda p_value: 'Reject005=0 : Heteroscedasticity' 
                                                                  if p_value < alpha else 'Reject005=1 : Homoscedasticity')
#two_tailed_new_show=two_tailed_new.drop(columns="greater")
two_tailed_new.astype(str)

,df0,df1,SSR0,SSR1,MSE0,MSE1,GQ_test_clusters,two_tailed,greater,Result_test
Replicate,,,,,,,,,,
2.0,8.0,16.0,3013220479462.5645,218848375189219.97,430460068494.65204,14589891679281.332,36.31469663120264,2.0805270697143996e-05,0.9999895973646514,Reject005=0 : Heteroscedasticity
3.0,8.0,16.0,10715074869154.88,46425326254765.5,1530724981307.84,3095021750317.7,2.16635566347784,0.2692298125572057,0.8653850937213972,Reject005=1 : Homoscedasticity
5.0,13.0,11.0,12519956169391.865,34773134466414.28,1043329680782.6554,3477313446641.428,3.282401471315456,0.04526687364198878,0.9773665631790056,Reject005=0 : Heteroscedasticity
10.0,15.0,9.0,43056547830838.06,63631091988283.08,3075467702202.7188,7953886498535.385,2.4630822795439054,0.11833554452172623,0.9408322277391369,Reject005=1 : Homoscedasticity
11.0,11.0,13.0,27432366480524.746,68502993838855.06,2743236648052.4746,5708582819904.589,2.1129810930074617,0.2216864581620024,0.8891567709189988,Reject005=1 : Homoscedasticity
13.0,9.0,15.0,1236820071312.9492,21982998399405.54,154602508914.11865,1570214171386.1099,10.664282821382145,0.001134965592211712,0.9994325172038941,Reject005=0 : Heteroscedasticity
16.0,16.0,8.0,159165376633038.88,139671159805872.55,10611025108869.258,19953022829410.363,1.7550445047843426,0.32193078649151086,0.8390346067542446,Reject005=1 : Homoscedasticity
19.0,16.0,8.0,2720455264304.037,121087496700350.88,181363684286.93582,17298213814335.84,89.02002417696671,1.6903145549918008e-11,0.9999999999915484,Reject005=0 : Heteroscedasticity
21.0,11.0,13.0,925891516687.5269,165218926687076.47,92589151668.75269,13768243890589.705,150.99029179339647,3.7658476337298907e-10,0.9999999998117076,Reject005=0 : Heteroscedasticity


In [9]:
# สร้างตารางผลลัพธ์
alpha = 0.05
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
two_tailed_new=pd.DataFrame({"Replicate":list_replicate, "df0":list_df0, "df1":list_df1, "SSR0":list_SSR0,
                     "SSR1":list_SSR1, "MSE0":list_MSE0, "MSE1":list_MSE1,
                     "GQ_test_clusters":list_GQ_test_clusters, "two_tailed":list_two_tailed, "greater":list_p_greater})
two_tailed_new.set_index(['Replicate'], inplace=True)

two_tailed_new["two_tailed_new"]=two_tailed_new[["two_tailed","greater"]].apply(lambda value: value[0] if value[0]<0.5 else 2*value[1], axis=1)
two_tailed_new["Result_test"]=two_tailed_new["two_tailed_new"].apply(lambda p_value: 'Reject005=0 : Heteroscedasticity' 
                                                                      if p_value < alpha else 'Reject005=1 : Homoscedasticity')


two_tailed_new_show=two_tailed_new.drop(columns=["two_tailed", "greater"])
two_tailed_new_show.astype(str)

,df0,df1,SSR0,SSR1,MSE0,MSE1,GQ_test_clusters,two_tailed_new,Result_test
Replicate,,,,,,,,,
0.0,11.0,13.0,36649167549782.555,36272380670693.35,3664916754978.2554,3022698389224.446,0.8374546126313229,0.7514958406325016,Reject005=1 : Homoscedasticity
1.0,10.0,14.0,47310326495717.9,391674874793.9958,5256702943968.655,30128836522.61506,0.005913460938286254,1.613407493761992e-12,Reject005=0 : Heteroscedasticity
2.0,8.0,16.0,3013220479462.5645,218848375189219.97,430460068494.65204,14589891679281.332,36.31469663120264,2.0805270697143996e-05,Reject005=0 : Heteroscedasticity
3.0,8.0,16.0,10715074869154.88,46425326254765.5,1530724981307.84,3095021750317.7,2.16635566347784,0.2692298125572057,Reject005=1 : Homoscedasticity
4.0,13.0,11.0,301634403000583.1,3234899060391.5723,25136200250048.594,323489906039.1572,0.012674490999323844,1.2700578771716293e-08,Reject005=0 : Heteroscedasticity
...,...,...,...,...,...,...,...,...,...
995.0,10.0,14.0,65241878817145.45,45910665482100.8,7249097646349.495,3531589652469.292,0.5026423683953876,0.2324459652583779,Reject005=1 : Homoscedasticity
996.0,17.0,7.0,243091072674502.7,159873820104.76788,15193192042156.418,26645636684.12798,0.0015971997137175494,3.545781039046685e-09,Reject005=0 : Heteroscedasticity
997.0,15.0,9.0,61947752488576.9,79110567342621.08,4424839463469.778,9888820917827.635,2.1284217792299778,0.1883787612530492,Reject005=1 : Homoscedasticity


In [10]:
two_tailed_new_show['Result_test'].value_counts()

Reject005=0 : Heteroscedasticity    636
Reject005=1 : Homoscedasticity      364
Name: Result_test, dtype: int64